# Cryptostock Analytica

In [11]:
# Initial imports
import os
import time
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from datetime import datetime, timedelta

%matplotlib inline

In [ ]:
# Load .env enviroment variables
load_dotenv(dotenv_path="Alpaca.env")

In [ ]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2"
)

In [ ]:
# Format current date as ISO format
today = pd.Timestamp("2023-04-12", tz="America/New_York").isoformat()

# Set the tickers
### CODE HERE

# Set timeframe to "1Day" for Alpaca API
timeframe = "1D"

# Get current closing prices for SPY and AGG
df_portfolio = api.get_bars(
    tickers,
    timeframe,
    start=today,
    end=today,
).df

In [13]:
def fetch_top_n_cryptos(n=10):
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        'vs_currency': 'usd',
        'order': 'market_cap_desc',
        'per_page': n,
        'page': 1,
        'sparkline': False,
    }
    response = requests.get(url, params=params)
    data = response.json()
    return data

def fetch_crypto_data(coin_id, from_timestamp, to_timestamp):
    url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart/range"
    params = {
        'vs_currency': 'usd',
        'from': from_timestamp,
        'to': to_timestamp
    }
    response = requests.get(url, params=params)
    data = response.json()
    df = pd.DataFrame(data['prices'], columns=['time', 'price'])
    df['time'] = pd.to_datetime(df['time'], unit='ms')
    return df

def get_data_by_name(name):
    for crypto_id, data in crypto_data.items():
        if crypto_id.lower() == name.lower():
            return data
    raise ValueError(f"No data found for '{name}'")

# Fetch the top 10 cryptocurrencies by market cap
top_cryptos = fetch_top_n_cryptos(10)

# Unix timestamps for three years of data
to_timestamp = pd.Timestamp.now(tz='UTC').timestamp()
from_timestamp = pd.Timestamp.now(tz='UTC') - pd.DateOffset(years=3)
from_timestamp = from_timestamp.timestamp()

# Fetch historical data for each of the top 10 cryptos
crypto_data = {}
for crypto in top_cryptos:
    coin_id = crypto['id']
    try:
        df = fetch_crypto_data(coin_id, from_timestamp, to_timestamp)
        crypto_data[coin_id] = df
        print(f"Fetched data for {coin_id}")
        time.sleep(2)  # Add a delay between requests to avoid rate limiting
    except Exception as e:
        print(f"Error fetching data for {coin_id}: {e}")

Fetched data for bitcoin
Fetched data for ethereum
Fetched data for tether
Fetched data for binancecoin
Fetched data for usd-coin
Fetched data for ripple
Fetched data for cardano
Fetched data for staked-ether
Fetched data for dogecoin
Fetched data for matic-network


In [ ]:
def get_data_by_name(name):
    for crypto in top_cryptos:
        if crypto['name'].lower() == name.lower():
            coin_id = crypto['id']
            return crypto_data[coin_id]
    return None

# Ask user for the name of the cryptocurrency
selected_crypto = input("Enter the name of the cryptocurrency you want to view data for: ")

# Get the data for the selected cryptocurrency
selected_crypto_data = get_data_by_name(selected_crypto)

if selected_crypto_data is not None:
    # Round the price data to 2 decimal places
    selected_crypto_data['price'] = selected_crypto_data['price'].round(2)

    # Print the data
    print(selected_crypto_data)
else:
    print("Error: Invalid cryptocurrency name.")